In [2]:
import numpy as np
import pandas as pd
from datasets import load_dataset
import os
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer
import torch

/home/lielbin/.conda/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
prefix_path = parent_dir

In [3]:
df_imdb_sub = pd.read_csv(os.path.join(prefix_path, "data", "imdb_text_subtitles.csv"))

In [4]:
df_imdb_sub.head()

,tconst_episode,tconst_season,seasonNumber,episodeNumber,title_episode,runtimeMinutes,title_series,averageRating_season,numVotes_series,averageRating_episode,numVotes_episode,Year_episode,sub_id,subtitles
0,tt0059753,tt0060028,1,0,The Cage,63.0,Star Trek,8.4,95360.0,7.6,7428.0,1966.0,188032.0,"הכלוב .תבדקו את המעגל- .הכול תקין, אדוני- .אם ..."
1,tt0061027,tt0060028,1,3,Where No Man Has Gone Before,50.0,Star Trek,8.4,95360.0,7.7,6688.0,1966.0,181335.0,",יומן הקפטן .זמן כוכבים 1312.4 ,הבלתי אפשרי קר..."
2,tt0066931,tt1466074,1,3,Dead Weight,76.0,Columbo,8.3,43873.0,6.9,3538.0,1971.0,94598.0,.דוטון .גנרל הוליסטר .היכנס ?רוצה לשתות- .לא. ...
3,tt0073677,tt0071054,3,17,The Secret of Bigfoot: Part 1,47.0,The Six Million Dollar Man,7.1,13511.0,7.6,399.0,1976.0,297110.0,***האיש השווה מיליונים*** תורגם סונכרן וקודד ע...
4,tt0084218,tt0083437,1,1,Knight of the Phoenix: Part 1,45.0,Knight Rider,6.9,38628.0,7.3,2272.0,1982.0,5919.0,"<i>- קונסוליה זרה, ארה""ב -</i> ?תסלחי לי לרגע ..."


In [6]:
example_subtitle = df_imdb_sub['subtitles'].iloc[0]
example_subtitle[:5000]

'הכלוב .תבדקו את המעגל- .הכול תקין, אדוני- .אם כך לא ייתכן שהבעיה במסך אין ספק שיש שם משהו .שמתקרב אלינו, קפטן .ייתכן שאלה המטאוראידים האלה .לא, זה משהו אחר. יש שם משהו בכל זאת .הוא מתקרב במהירות האור .במסלול התנגשות קרן המטאוראידים .אינה מסיטה אותו ממסלולו, קפטן ?לבצע תמרון התחמקות, אדוני- .נישאר במסלולנו- .זה גל רדיו, אדוני .אנחנו חולפים בתוך אות מצוקה בסגנון ישן האותות תוכנתו לגרום הפרעות .ולמשוך תשומת לב לאזור הזה ,ספינת חלל במצוקה מבצעת נחיתת אונס .אדוני .זה הכול, אין הודעות נוספות- .איתרתי את המיקום- .האות מגיע מקבוצת כוכבי טאלוס אין לנו ספינות או מושבות של כדור הארץ .במרחק כזה ,לפי האות מדובר במשלחת מחקר ששמה .אס-אס קולומביה .היא נעלמה באזור הזה לפני 18 שנה בערך .זה הזמן שיידרש לקרן רדיו להגיע משם לכאן ,לפי מאגר הנתונים .קבוצת טאלוס מעולם לא נחקרה מערכת השמש דומה לזו של כדור הארץ .וכוללת 11 כוכבי לכת .נראה שכוכב הלכת הרביעי הוא מסוג אם .בעל אטמוספירת חמצן ,אז ייתכן שהם עדיין בחיים .אפילו לאחר 18 שנה .אם הם שרדו את ההתרסקות ?לא נצא לשם כדי לוודא .לא בלי סימנים לכך שישנם ניצולים ת

## Embedding to the first 512 tokens

In [12]:
# Load the Hebrew model and tokenizer
model_name = "dicta-il/dictabert-lex"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

2024-12-17 17:16:07.727056: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734448568.227718 2504646 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734448568.333372 2504646 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-17 17:16:10.015094: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of BertModel were not initialized from the model checkpoint at dicta-il/dictabert-lex and are newly initialized:

In [21]:
# Set the device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(128000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [67]:
def get_long_text_embeddings(text, max_length=512, overlap=100):
    # Tokenize the text
    tokens = tokenizer.encode(text, add_special_tokens=False)
    
    # Split into chunks
    chunks = []
    for i in range(0, len(tokens), max_length - overlap):
        chunk = tokens[i:i + max_length]
        chunks.append(chunk)
    
    # Generate embeddings for each chunk
    chunk_embeddings = []
    for chunk in chunks:
        # Prepare the chunk
        chunk = [tokenizer.cls_token_id] + chunk + [tokenizer.sep_token_id]
        
        # Pad or truncate to exact max_length
        if len(chunk) > max_length:
            chunk = chunk[:max_length]
        elif len(chunk) < max_length:
            chunk = chunk + [tokenizer.pad_token_id] * (max_length - len(chunk))
        
        # Convert to tensor and move to device
        input_ids = torch.tensor([chunk]).long().to(device)
        
        # Generate embedding
        with torch.no_grad():
            outputs = model(input_ids)
            chunk_embedding = outputs.last_hidden_state.mean(dim=1)
        
        chunk_embeddings.append(chunk_embedding.cpu())
    
    # Average chunk embeddings
    final_embedding = torch.cat(chunk_embeddings, dim=0).mean(dim=0)
    
    return final_embedding

In [63]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [68]:
# Step 1: Split the text into chunks
# example_subtitle_chunks = split_text_into_chunks(example_subtitle, tokenizer, max_length=512, overlap=50)

# Step 2: Generate embeddings and aggregate
final_embedding = get_long_text_embeddings(example_subtitle)


In [69]:
final_embedding.shape

torch.Size([768])

In [72]:
df_imdb_sub_embedding = df_imdb_sub.copy()

In [73]:
embeddings = []

# Use tqdm for progress bar
tqdm.pandas()

# Generate embeddings for subtitles
def generate_embedding(text):
    try:
        return get_long_text_embeddings(text)
    except Exception as e:
        print(f"Error processing text: {e}")
        return None

# Apply embedding generation with progress bar
df_imdb_sub_embedding['embedding_subtitles'] = df_imdb_sub_embedding['subtitles'].progress_apply(generate_embedding)

# Optional: Remove rows where embedding failed
df_imdb_sub_embedding = df_imdb_sub_embedding.dropna(subset=['embedding_subtitles'])

# Optional: Convert embeddings to a storable format if needed
# This converts torch tensor to list for easier storage
df_imdb_sub_embedding['embedding_subtitles'] = df_imdb_sub_embedding['embedding_subtitles'].apply(lambda x: x.tolist())

In [75]:
# Ensure the function is defined with the parameters
def gpu_batch_embeddings(df, 
                         column, 
                         model, 
                         tokenizer, 
                         max_length=512, 
                         batch_size=32):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    embeddings = []
    
    for i in tqdm(range(0, len(df), batch_size)):
        batch = df[column].iloc[i:i+batch_size].tolist()
        
        # Tokenize batch
        inputs = tokenizer(batch, 
                           return_tensors="pt", 
                           max_length=max_length, 
                           truncation=True, 
                           padding='max_length')
        
        # Move to GPU
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Generate embeddings
        with torch.no_grad():
            outputs = model(**inputs)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1)
        
        # Move back to CPU and convert to list
        batch_embeddings = batch_embeddings.cpu().numpy().tolist()
        embeddings.extend(batch_embeddings)
    
    df_embedded = df.copy()
    df_embedded['embedding_subtitles'] = embeddings
    
    return df_embedded

In [76]:
df_imdb_sub_embedding = gpu_batch_embeddings(
    df=df_imdb_sub,
    column='subtitles',
    model=model,
    tokenizer=tokenizer,
    batch_size=32,  # Adjust based on your GPU memory
    max_length=512  # Adjust based on your model's max sequence length
)

100%|██████████| 2098/2098 [18:53<00:00,  1.85it/s]


In [77]:
df_imdb_sub_embedding.head()

,tconst_episode,tconst_season,seasonNumber,episodeNumber,title_episode,runtimeMinutes,title_series,averageRating_season,numVotes_series,averageRating_episode,numVotes_episode,Year_episode,sub_id,subtitles,embedding_subtitles
0,tt0059753,tt0060028,1,0,The Cage,63.0,Star Trek,8.4,95360.0,7.6,7428.0,1966.0,188032.0,"הכלוב .תבדקו את המעגל- .הכול תקין, אדוני- .אם ...","[0.09006182104349136, -0.2422049343585968, -0...."
1,tt0061027,tt0060028,1,3,Where No Man Has Gone Before,50.0,Star Trek,8.4,95360.0,7.7,6688.0,1966.0,181335.0,",יומן הקפטן .זמן כוכבים 1312.4 ,הבלתי אפשרי קר...","[-0.09532366693019867, -0.24570876359939575, -..."
2,tt0066931,tt1466074,1,3,Dead Weight,76.0,Columbo,8.3,43873.0,6.9,3538.0,1971.0,94598.0,.דוטון .גנרל הוליסטר .היכנס ?רוצה לשתות- .לא. ...,"[0.07532934844493866, -0.2646660804748535, -0...."
3,tt0073677,tt0071054,3,17,The Secret of Bigfoot: Part 1,47.0,The Six Million Dollar Man,7.1,13511.0,7.6,399.0,1976.0,297110.0,***האיש השווה מיליונים*** תורגם סונכרן וקודד ע...,"[-0.08691376447677612, -0.1898632049560547, -0..."
4,tt0084218,tt0083437,1,1,Knight of the Phoenix: Part 1,45.0,Knight Rider,6.9,38628.0,7.3,2272.0,1982.0,5919.0,"<i>- קונסוליה זרה, ארה""ב -</i> ?תסלחי לי לרגע ...","[-0.07392537593841553, -0.1274789273738861, -0..."


In [78]:
df_imdb_sub_embedding.to_csv(os.path.join(prefix_path, "data", "imdb_embedding_subtitles.csv"), index=False)